## KMeans - TimeDivisionKMeans 비교

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import warnings
import random as ran

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings(action='ignore') 

from src import crs, PublicPredictor, KMeans, TimeDivisionKMeans
from src.dbc import utils
import src.utils as ut
from IPython.display import clear_output

In [2]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)

# 3. data init
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 30
APT = crs.utils.get_APT(month_df, PUBLIC_PERCENTAGE)

calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
apt = calc.apart

In [3]:
m_15 = utils.data_preprocessing(xlsx)

df = utils.dimension_reduction(m_15)
m_60 = df.copy()

m_60.head()

m_60_1 = m_60[m_60.index.month == 1].copy()
m_60_1.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2019-01-01 00:00:00,0.033,0.000,0.034,0.037,0.062,0.322,0.240,0.295,0.373,0.243,...,0.534,0.188,0.400,0.318,0.420,0.626,0.484,0.289,0.305,0.652
2019-01-01 01:00:00,0.048,0.000,0.033,0.037,0.063,0.185,0.257,0.397,0.257,0.228,...,0.396,0.434,0.355,0.250,0.465,0.409,0.459,0.336,0.332,0.557
2019-01-01 02:00:00,0.032,0.000,0.039,0.036,0.062,0.175,0.384,0.353,0.123,0.141,...,0.194,0.337,0.379,0.274,0.326,0.307,0.405,0.382,0.311,0.491
2019-01-01 03:00:00,0.033,0.000,0.039,0.037,0.062,0.167,0.276,0.488,0.142,0.159,...,0.286,0.263,0.375,0.264,0.336,0.345,0.298,0.291,0.261,0.511
2019-01-01 04:00:00,0.032,0.001,0.033,0.037,0.063,0.197,0.266,0.278,0.190,0.227,...,0.275,0.245,0.310,0.423,0.360,0.364,0.212,0.347,0.294,0.507


In [29]:
kmeans = KMeans(datas=m_60_1.T.values, ver=3)
kmeans.fit()
kmeans_group_df = ut.make_group_df(m_60_1, kmeans)
ut.get_anomaly_df(kmeans_group_df)

ECV : 47 %


,가구명,usage (kWh),label
44,아파트1-102-801,280,0
50,아파트1-103-1006,284,0
11,아파트1-103-1602,387,1
13,아파트1-103-1502,397,1
14,아파트1-103-202,319,1
17,아파트1-102-203,326,1
23,아파트1-101-1602,350,1
28,아파트1-101-303,376,1
30,아파트1-101-1403,380,1
36,아파트1-101-504,322,1


In [34]:
kmeans_group_df[kmeans_group_df['label'] == 2]

,가구명,usage (kWh),label
20,아파트1-102-403,424,2
27,아파트1-101-402,345,2
34,아파트1-101-901,297,2
37,아파트1-102-104,309,2
43,아파트1-102-802,406,2
46,아파트1-103-1805,386,2
51,아파트1-103-1903,405,2
65,아파트1-103-203,371,2
80,아파트1-103-1002,494,2
87,아파트1-101-804,316,2


In [33]:
kmeans_group_df[kmeans_group_df['label'] == 3]

,가구명,usage (kWh),label
38,아파트1-102-301,416,3
62,아파트1-103-703,498,3
81,아파트1-102-404,436,3


In [ ]:
kmeans_test = pd.DataFrame(columns=['anomaly 발생 수', '아웃라이어 군집'])
tdkmeans_test = pd.DataFrame(columns=['anomaly 발생 수', '아웃라이어 군집'])

for case in range(0, 500):
    test_m = m_60_1.sample(n=ran.randrange(50, len(m_60_1.columns)), axis=1).copy()

    kmeans = KMeans(datas=test_m.T.values, ver=3)
    kmeans.fit()
    kmeans_group_df = ut.make_group_df(test_m, kmeans)

    tdkmeans = TimeDivisionKMeans(datas=test_m)
    tdkmeans.fit()
    tdkmeans_group_df = ut.make_group_df(test_m, tdkmeans, _type="tdkmeans")

    kmeans_test = kmeans_test.append({
        "anomaly 발생 수":len(ut.get_anomaly_df(kmeans_group_df)),
        "아웃라이어 군집":np.where(kmeans_group_df['label'].value_counts().values == 1)[0].size
    }, ignore_index=True)
    tdkmeans_test = tdkmeans_test.append({
        "anomaly 발생 수":len(ut.get_anomaly_df(tdkmeans_group_df)),
        "아웃라이어 군집":np.where(tdkmeans_group_df['label'].value_counts().values == 1)[0].size
    }, ignore_index=True)
    
    clear_output(wait=True)

ECV : 49 %
1/248 - ECV:86%
11/248 - ECV:80%
21/248 - ECV:87%
31/248 - ECV:90%
41/248 - ECV:88%
51/248 - ECV:85%
61/248 - ECV:88%
71/248 - ECV:86%
81/248 - ECV:87%
91/248 - ECV:85%
101/248 - ECV:87%
111/248 - ECV:83%
121/248 - ECV:89%
131/248 - ECV:81%
141/248 - ECV:86%
151/248 - ECV:89%
161/248 - ECV:89%
171/248 - ECV:79%
181/248 - ECV:81%
191/248 - ECV:83%
201/248 - ECV:90%
211/248 - ECV:82%
221/248 - ECV:86%
231/248 - ECV:80%
241/248 - ECV:89%
248/248 - ECV:87%


In [ ]:
kmeans_test.mean()

In [ ]:
tdkmeans_test.mean()

In [ ]:
print(kmeans_test['anomaly 발생 수'].max(), kmeans_test['아웃라이어 군집'].max())

In [ ]:
print(tdkmeans_test['anomaly 발생 수'].max(), tdkmeans_test['아웃라이어 군집'].max())

In [ ]:
print(kmeans_test['anomaly 발생 수'].min(), kmeans_test['아웃라이어 군집'].min())

In [ ]:
print(tdkmeans_test['anomaly 발생 수'].min(), tdkmeans_test['아웃라이어 군집'].min())

In [ ]:
kmeans_test['anomaly 발생 수'].value_counts()

In [ ]:
tdkmeans_test['anomaly 발생 수'].value_counts()

In [ ]:
kmeans_test['아웃라이어 군집'].value_counts()

In [ ]:
tdkmeans_test['아웃라이어 군집'].value_counts()

In [15]:
kmeans_test = pd.DataFrame(columns=['anomaly 발생 수', '아웃라이어 군집'])
tdkmeans_test = pd.DataFrame(columns=['anomaly 발생 수', '아웃라이어 군집'])

for case in range(0, 500):
    test_m = m_60_1.sample(n=ran.randrange(50, len(m_60_1.columns)), axis=1).copy()

    kmeans = KMeans(datas=test_m.T.values, ver=2)
    kmeans.fit()
    kmeans_group_df = ut.make_group_df(test_m, kmeans)

    tdkmeans = TimeDivisionKMeans(datas=test_m)
    tdkmeans.fit()
    tdkmeans_group_df = ut.make_group_df(test_m, tdkmeans, _type="tdkmeans")

    kmeans_test = kmeans_test.append({
        "anomaly 발생 수":len(ut.get_anomaly_df(kmeans_group_df)),
        "아웃라이어 군집":np.where(kmeans_group_df['label'].value_counts().values == 1)[0].size
    }, ignore_index=True)
    tdkmeans_test = tdkmeans_test.append({
        "anomaly 발생 수":len(ut.get_anomaly_df(tdkmeans_group_df)),
        "아웃라이어 군집":np.where(tdkmeans_group_df['label'].value_counts().values == 1)[0].size
    }, ignore_index=True)
    
    clear_output(wait=True)

ECV : 58 %
1/248 - ECV:87%
11/248 - ECV:77%
21/248 - ECV:84%
31/248 - ECV:88%
41/248 - ECV:86%
51/248 - ECV:82%
61/248 - ECV:91%
71/248 - ECV:82%
81/248 - ECV:87%
91/248 - ECV:84%
101/248 - ECV:84%
111/248 - ECV:83%
121/248 - ECV:90%
131/248 - ECV:76%
141/248 - ECV:86%
151/248 - ECV:87%
161/248 - ECV:87%
171/248 - ECV:77%
181/248 - ECV:82%
191/248 - ECV:80%
201/248 - ECV:89%
211/248 - ECV:82%
221/248 - ECV:85%
231/248 - ECV:79%
241/248 - ECV:88%
248/248 - ECV:84%
